In [1]:
### importing packages
import os
import json
import random
import time  # to time execution time
import torch  # ML lib. allows computation of tensors on GPUs
import pandas as pd  # data analysis and manipulation
import numpy as np  # for mathematical operations
import matplotlib.pyplot as plt  # plotting

plt.style.use("ggplot")
from tqdm.notebook import tqdm  # for progress bars
from transformers import BertTokenizer  # tokenizes sequences of strings
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import RandomSampler

# create tensors from data and make them accesible for model
from transformers import BertForSequenceClassification
from transformers import (
    AdamW,
    get_linear_schedule_with_warmup,
)

OSError: [WinError 127] Die angegebene Prozedur wurde nicht gefunden. Error loading "C:\Users\NALMPI\anaconda3\lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.

In [2]:
# Read data
data = pd.read_csv("... .csv".format(dataset))

# sample size 
n = len(data)

# first observations
data.head()

NameError: name 'pd' is not defined

In [3]:
# store model specifications and device name for use in evaluation.py
model_no = 0

# Try 4: 
# bert-base-german-cased (trained by Deepset.ai)
# bert-base-german-dbmdz-cased (trained by by DBMDZ)
# bert-base-german-dbmdz-uncased (trained by by DBMDZ)
# distilbert-base-german-cased (distilled from DBMDZ)
pretrained_model = "bert-base-german-cased"

# hyperparameters: max length, batch size; 
# optimizer parameters: learning rate, tolerance;
# number of epochs

max_length = 128
batch_size = 16
lr = 1e-5
eps = 1e-8
epochs = 4
device_name = "n/a"

model_parameters = {
    "model_type": pretrained_model,
    "MAX_LEN": max_length,
    "BATCH_SIZE": batch_size,
    "LR": lr,
    "EPS": eps,
    "device_name": device_name, # for GPU
}

NameError: name 'device_name' is not defined

In [4]:
# Preprocessing
# Tokenization 

tokenizer = BertTokenizer.from_pretrained(pretrained_model)
# , do_lower_case=True for uncased 

NameError: name 'BertTokenizer' is not defined

In [5]:
# encode textual data 
encoded_train = tokenizer.batch_encode_plus(
    data[full_text].values,     
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=max_len,
    truncation=True,
    return_tensors="pt",
)

# extract model input
input_ids_train = encoded_train["input_ids"]
att_masks_train = encoded_train["attention_mask"]
labels_train = torch.tensor(df[label].values, dtype=torch.long)

# TensorDataset combines prepared model inputs in a tensor to train
# the model
tensor_dataset_train = TensorDataset(
    input_ids_train, att_masks_train, labels_train
)

num_labels = len(data[labels].unique())

NameError: name 'tokenizer' is not defined

In [6]:
# Put into a classification task
model = BertForSequenceClassification.from_pretrained(
    pretrained_model,
    num_labels=num_labels,
    output_attentions=False,
    output_hidden_states=True, # to extract the embeddings
)

# DataLoader helps the model to read in the data
dataloader_train = DataLoader(
    tensor_dataset_train, sampler = RandomSampler(tensor_dataset_train), batch_size = batch_size
)

NameError: name 'BertForSequenceClassification' is not defined

In [2]:
# Optimizer ADAM - define
optimizer = AdamW(model.parameters(), lr=lr, eps=eps)

# learning rate schedule
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train) * epochs,
)

NameError: name 'AdamW' is not defined

In [ ]:
# seed
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
# to avoid non-determinism
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


# Attach model to GPU device: 

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# with tf you can retrieve the device name (may lead to problems when also using pytorch)
# try:
#    device_name = tf.test.gpu_device_name()
# except:
#    device_name = 'n/a'


model.to(device)
start = time.time()
train_loss_all_epochs = []
f1_score_all_epochs = []
train_loss_set = []


# training loop, tqdm for progress bar
for epoch in tqdm(range(1, epochs + 1)):
    # set model to train mode
    model.train()
    # initiate cumulated train loss
    loss_train_total = 0
    # define progress bar (to loop over dataloader in progress bar)
    progress_bar = tqdm(
        dataloader_train,
        desc="Epoch {:1d}".format(epoch),
        leave=False,
        disable=False,
    )
    for batch in progress_bar:
        # clear gradients from memory
        model.zero_grad()
        # add batches to GPU
        batch = tuple(b.to(device) for b in batch)
        # Unpack the inputs from dataloader
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "labels": batch[2],
        }
        # forward pass: calculate logit predictions
        outputs = model(**inputs)
        # store batch loss
        loss = outputs[0]
        # track batch loss
        train_loss_set.append(loss.item())
        # accumulate batch loss
        loss_train_total += loss.item()
        # backward pass
        loss.backward()
        # gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        # print progress (training loss of current epoch)
        progress_bar.set_postfix(
            {"training_loss": "{:.3f}".format(loss.item() / len(batch))}
        )

    # plot training performance
    plt.figure(figsize=(15, 8))
    plt.title("Training loss")
    plt.xlabel("Batch")
    plt.ylabel("Loss")
    plt.plot(train_loss_set)
    # plt.show()
    plt.savefig(
        "./results/sentiment_analysis/training/{}/train_loss_{}_epoch_{}.png".format(
            dataset, model_no, epoch
        )
    )

    # plot training performance last 50 batches
    k = n / BATCH_SIZE
    k2 = int(k - 1)
    k1 = int(k - 50)
    plt.figure(figsize=(15, 8))
    plt.title("Training Loss - Last 50 Batches")
    plt.xlabel("Batch")
    plt.ylabel("Loss")
    plt.plot(train_loss_set[k1:k2])
    # plt.show()
    plt.savefig(
        "./results/sentiment_analysis/training/{}/train_loss_50_{}_epoch_{}.png".format(
            dataset, model_no, epoch
        )
    )

    # save model after each epoch
    torch.save(
        model.state_dict(),
        f"./results/sentiment_analysis/training/{dataset}/BERT_{model_no}_epoch_{epoch}.model",
    )

    # print current epoch
    tqdm.write(f"\nEpoch {epoch}")

    # compute train loss (cumulated loss/number of batches)
    loss_train_avg = loss_train_total / len(dataloader_train)

    # print train loss
    tqdm.write(f"Training loss: {loss_train_avg}")

end = time.time()
wall_time = end - start
print("wall_time: ", wall_time, "s")